In [1]:
#Theory Behind HMM (Jurafsky Notes):
""" A Markov chain is useful when we need to compute a probability for a sequence of
events that we can observe in the world. In many cases, however, the events we are
interested in may not be directly observable in the world. For example, in part-of-
speech tagging, we don’t observe part-of-speech tags in the world; we see words and have to infer the correct 
tags from the word sequence. Hence, we call the part-of-speech tags hidden because they are not observed.
"""
"""Imagine that you are a climatologist in the year 2799 studying the history of global
warming. You cannot find any records of the weather in Baltimore, Maryland, for
the summer of 2007, but you do find Jason Eisner’s diary, which lists how many ice
creams Jason ate every day that summer. Our goal is to use these observations to
estimate the temperature every day. We’ll simplify this weather task by assuming
there are only two kinds of days: cold (C) and hot (H). So the Eisner task is as
follows:
Given a sequence of observations O, each observation an integer cor-
responding to the number of ice creams eaten on a given day, figure
out the correct ‘hidden’ sequence Q of weather states (H or C) which
caused Jason to eat the ice cream."""
""""""

''

In [2]:
#Terminology
""" 
Q = q1 q2 . . . qN             : Set of N states
A = a[i][j] s.t. 1<=i,j<=N     : Transition probability matrix A,s.t a[i][j] is prob of moving from state i to j
a[i][j]=P(qj|qi)
pi = pi[1],pi[2],...,pi[N]     : Initial prob distr over states. pi[i] is prob of Markov chain starting in state i.
O = o[1]o[2]...O[T]            : A Sequence of T Observations, drawn from a Vocab V=v[1]v[2]...,v[v]
"""
""""""

''

In [3]:
#Assumptions:
"""
I have Assumped the Observations,States to be 0,1,2.... i.e Natural Numbers:
So, please make sure to encode textual/categorical/arbitrary numeric data to Continuous Integers starting from 0 
"""
""""""

''

In [4]:
"""A first-order hidden Markov model instantiates two simplifying assumptions.
First, as with a first-order Markov chain, the probability of a particular state depends
only on the previous state:
    Markov Assumption: P(qi |q1 ...qi−1) = P(qi|qi−1)
Second, the probability of an output observation o[i] depends only on the state that
produced the observation qi and not on any other states or any other observations:
    Output Independence: P(o[i]|q1 . . qi , . . , qT , o[1] , . . , o[i] , . . , o[T])= P(o[i]|qi)"""
""""""

''

In [5]:
"""There is a (non-zero) probability of transitioning between any two states. Such an HMM is called 
a fully connected or ergodic HMM. Sometimes, however, we have HMMs in which many of the transitions 
between states have zero probability."""
""""""

''

In [6]:
"""hidden Markov models should be characterized by three fundamental problems:
    Problem 1 (Likelihood): Given an HMM λ = (A, B) and an observation sequence O, determine the 
    likelihood P(O|λ ).
    Problem 2 (Decoding): Given an observation sequence O and an HMM λ =(A, B), discover the best hidden state 
    sequence Q.
    Problem 3 (Learning): Given an observation sequence O and the set of states in the HMM, learn the HMM 
    parameters A and B."""
""""""

''

In [7]:
import numpy as np
import math
from math import log2
import sys

In [8]:
#Likelihood Computation: The Forward Algorithm  O(N*N*T)
#Our first problem is to compute the likelihood of a particular observation sequence say 3 1 3?
"""For a Markov chain, where the surface observations are the same as the hidden
events, we could compute the probability of 3 1 3 just by following the states labeled
3 1 3 and multiplying the probabilities along the arcs. For a hidden Markov model,
things are not so simple. We want to determine the probability of an ice-cream
observation sequence like 3 1 3, but we don’t know what the hidden state sequence
is!"""
#O: Observations, a:State Transition Prob, b:Emission Prob, pi: Initial Prob. 
#a[i][j]: Prob of transition from state i to state j
#b[i][j]: prob of emitting observation j at state i
#pi[i]: Initial Prob.of state i
#P(O|a,b,pi)=P(O|X=x1,a,b,pi)+P(O|X=x2,a,b,pi)+......+P(O|X=xn,a,b,pi) s.t. x1,x2,....,xn are all possible sequences
#alpha[t][i]:Partial Observation sequence upto time t is generated and we are state i
#alpha[t][i]= alpha[t-1][0]*a[0][i]*b[i][o[t]]+
#    alpha[t-1][1]*a[1][i]*b[i][o[t]]+....+alpha[t-1][n-1]*a[n-1][i]*b[i][o[t]]#Assuming we have n states
def computeAlpha(observations,a,b,pi,alphaDP):
    statesC=a.shape[0]
    timePts=observations.shape[0]
    if timePts<1:
        return
    for i in np.arange(statesC):
        alphaDP[0][i]=pow(2,log2(pi[i])+log2(b[i][observations[0]]))#pi[i]*b[i][observations[0]]
    for t in np.arange(1,timePts):
        for i in np.arange(statesC):
            for j in np.arange(statesC):
                alphaDP[t][i]+=pow(2,log2(alphaDP[t-1][j])+log2(a[j][i])+log2(b[i][observations[t]]))
                #alphaDP[t][i]=alphaDP[t][i]-(log10(alphaDP[t-1][j])+log10(a[j][i])+log10(b[i][observations[t]]))
                #alphaDP[t][i]+=alphaDP[t-1][j]*a[j][i]*b[i][observations[t]]
def observationProb(observations,a,b,pi,alphaDP):
    computeAlpha(observations,a,b,pi,alphaDP)
    print(alphaDP)
    timePts=observations.shape[0]
    stateC=a.shape[0]
    ans=0.0
    for i in np.arange(stateC):
        ans+=alphaDP[timePts-1][i]
    return ans
def observationsLikelihood(alphaDP):
    timePts=alphaDP.shape[0]
    stateC=alphaDP.shape[1]
    ans=0.0
    for i in np.arange(stateC):
        ans+=alphaDP[timePts-1][i]
    return ans

In [9]:
observations=np.array([0,1,0,2,0,1,2])
A=np.array([[0.7,0.3],[0.4,0.6]])
B=np.array([[0.1,0.4,0.5],[0.7,0.2,0.1]])
pi=np.array([0.6,0.4])
alphaDP=np.zeros(shape=(observations.shape[0],A.shape[0]))# Count_of_Observations*Count_of_Hidden_States
#Can Easily Optimize Space Requirement of above alphaDP; Only need 2*count_of_Hidden_States Space
print(observationProb(observations,A,B,pi,alphaDP))
print(observationsLikelihood(alphaDP))

[[  6.00000000e-02   2.80000000e-01]
 [  6.16000000e-02   3.72000000e-02]
 [  5.80000000e-03   2.85600000e-02]
 [  7.74200000e-03   1.88760000e-03]
 [  6.17444000e-04   2.41861200e-03]
 [  5.59862240e-04   3.27280080e-04]
 [  2.61407800e-04   3.64326720e-05]]
0.000297840472
0.000297840472


In [10]:
"""Problem 2 (Decoding): The Viterbi Algorithm -
Decoding: Given as input an HMM λ = (A, B, pi) and a sequence of observations O = o1 , o2 , ..., oT
find the most probable sequence of states Q = q1 q2 q3 . . . qT 
"""
"""Note that the Viterbi algorithm is identical to the forward algorithm except that it takes the max over the
previous path probabilities whereas the forward algorithm takes the sum. Note also
that the Viterbi algorithm has one component that the forward algorithm doesn’t have: backpointers. The reason 
is that while the forward algorithm needs to produce an observation likelihood, the Viterbi algorithm must 
produce a probability and also the most likely state sequence. We can compute this best state sequence by keeping
track of the path of hidden states that led to each state, and then at the end backtracing 
the best path to the beginning (the Viterbi backtrace).
"""
def ViterbiAlgo(observations,a,b,pi,viterbiDP,viterbiBackPtr):
    statesC=a.shape[0]
    timePts=observations.shape[0]
    if statesC<1:
        return
    for i in np.arange(statesC):
        viterbiDP[0][i]=pow(2,log2(pi[i])+log2(b[i][observations[0]]))#pi[i]*b[i][observations[0]]
    #Doesn't Handle the Case when there is single observation(1 Time Point)
    for t in np.arange(1,timePts):
        for i in np.arange(statesC):
            maxSoFar=0
            maxJ=-1
            for j in np.arange(statesC):
                tempVal=pow(2,log2(viterbiDP[t-1][j])+log2(a[j][i]))#viterbiDP[t-1]*a[j][i]
                if tempVal>maxSoFar:
                    maxSoFar=tempVal
                    maxJ=j
            viterbiDP[t][i]=tempVal*b[i][observations[t]]
            #At State i,Produce observation b[i][observations[t]]
            viterbiBackPtr[t][i]=maxJ
def getOptimalHiddenStates(observations,a,b,pi,viterbiDP,viterbiBackPtr):
    ViterbiAlgo(observations,a,b,pi,viterbiDP,viterbiBackPtr)
    print(viterbiDP)
    print(viterbiBackPtr)
    timePts=observations.shape[0]
    stateC=a.shape[0]
    ans=0.0
    endState=-1
    for i in np.arange(stateC):
        if ans<alphaDP[timePts-1][i]:
            ans=alphaDP[timePts-1][i]
            endState=i
    stateSeq=np.zeros(shape=(timePts))
    stateSeq[timePts-1]=endState#This is where we are adding the Final Optimal State for the Given Observation
    i=timePts-2
    prevState=endState
    while i>=0:
        prevState=viterbiBackPtr[i+1][prevState]
        stateSeq[i]=prevState
        i=i-1
    return {"Probability":ans,"Sequence":stateSeq}

In [11]:
observations=np.array([0,1,0,2])
A=np.array([[0.7,0.3],[0.4,0.6]])
B=np.array([[0.1,0.4,0.5],[0.7,0.2,0.1]])
pi=np.array([0.6,0.4])
viterbiDP=np.zeros(shape=(observations.shape[0],A.shape[0]))# Count_of_Observations*Count_of_Hidden_States
viterbiBackPtr=np.zeros(shape=(observations.shape[0],A.shape[0]))# Count_of_Observations*Count_of_Hidden_States
getOptimalHiddenStates(observations,A,B,pi,viterbiDP,viterbiBackPtr)

[[ 0.06        0.28      ]
 [ 0.0448      0.0336    ]
 [ 0.001344    0.014112  ]
 [ 0.0028224   0.00084672]]
[[ 0.  0.]
 [ 1.  1.]
 [ 0.  1.]
 [ 1.  1.]]


/home/alok/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:50: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


{'Probability': 0.0077420000000000067, 'Sequence': array([ 1.,  1.,  1.,  0.])}

In [12]:
"""Problem 3 (HMM Parameter Training): The Forward-Backward/Baum-Welch Algorithm(Special Case of EM Algo) -
Given an observation sequence O and the set of possible states in the HMM, learn the HMM 
parameters λ = (A, B) 
"""
"""The input to such a learning algorithm would be an unlabeled sequence of ob-
servations O and a vocabulary of potential hidden states Q. Thus, for the ice cream
task, we would start with a sequence of observations O = {1, 3, 2, ..., } and the set
of hidden states H and C. For the part-of-speech tagging task, we would start with
a sequence of observations O = {w1 , w2 , w3 . . .} and a set of hidden states NN, NNS,
VBD, IN,... and so on.
"""
"""
Let us begin by considering the much simpler case of training a Markov chain rather than a hidden Markov model. Since 
the states in a Markov chain are observed, we can run the model on the observation sequence and directly see which path 
we took through the model and which state generated each observation symbol. A Markov chain of course has no emission 
probabilities B (alternatively, we could view a Markov chain as a degenerate hidden Markov model where all the b proba-
bilities are 1.0 for the observed symbol and 0 for all other symbols). Thus, the only probabilities we need to train are 
the transition probability matrix A.
"""
"""
We get the maximum likelihood estimate of the probability a[i][j] of a particular transition between states i and j by 
counting the number of times the transition was taken, lets say C(i → j), and then normalizing by the total count of all
times we took any transition from state i:
    a[i][j]=C(i → j)/(Sum(C(i → q))forAll q∈Q)
"""
""""""

''

In [13]:
"""
For an HMM, we cannot compute these counts directly from an observation sequence since we don’t know which path of states 
was taken through the machine for a given input. The Baum-Welch algorithm uses two neat intuitions to solve this problem.
The first idea is to iteratively estimate the counts. We will start with an estimate for the transition and observation 
probabilities and then use these estimated probabilities to derive better and better probabilities. The second idea is 
that we get our estimated probabilities by computing the forward probability for an observation and then dividing that 
probability mass among all the different paths that contributed to this forward probability.
"""
""""""

''

In [14]:
"""
Lets define a useful probability related to the forward probability, called as Backward Probability(β).
β[t][i]: Probability of seeing the observations from t + 1 to end(T), given 
            that we are in state i at time t (and given λ )
β[t][i] = P(o[t+1] , o[t+2] . . . o[T] |q[t] = i, λ=(A,B) )
"""
""""""

''

In [15]:
"""
Similar to alpha, beta also follows the Recursive SubProperty, containing lot of recomputations
beta[t][i]= a[i][0]*b[0][o[t+1]]*beta[t+1][0]+a[i][1]*b[1][o[t+1]]*beta[t+1][1]+...+
                a[i][N-2]*b[t+1][o[t+1]]*beta[t+1][N-2]+a[i][N-1]*b[N-1][o[t+1]]*beta[t+1][N-1]
#Assuming we have n states, from 0 to n-1
"""
""""""
def computeBeta(observations,a,b,pi,betaDP):
    statesC=a.shape[0]
    timePts=observations.shape[0]
    if timePts<1:
        return
    for state in np.arange(statesC):
            betaDP[timePts-1][state]=1
    for t in np.arange(timePts-2,-1,-1):
        for i in np.arange(statesC):
            for j in np.arange(statesC):
                #betaDP[t][i]=betaDP[t][i]+a[i][j]*b[j][observations[t+1]]*betaDP[t+1][j]
                betaDP[t][i]+=pow(2,log2(a[i][j])+log2(b[j][observations[t+1]])+log2(betaDP[t+1][j]))
    return betaDP

In [16]:
#Compute Transition Probability: a[i][j]
"""
a[i][j]=Expected No of Transitions from State i to State j/Expected No of Transitions from State i=aNum(i,j)/aDenom(i)
diGamma(t,i,j) : Prob. of being in state i at time t and state j at time t+1
diGamma(t,i,j)=P(q[t]=i,q[t+1]=j|O,λ)=p(q[t]=i,q[t+1]=j,O|λ)/P(O|λ)=diGammaNum(t,i,j)/diGammaDenom()
diGammaNum(t,i,j)=p(q[t]=i,q[t+1]=j,O|λ)=alphaDP[t][i]*a[i][j]*b[j][observations[t+1]]*betaDP[t+1][j]
diGammaDenom=P(O|λ)=observationsLikelihood(alphaDP) i.e. Sum of alphaDP[t-1][i] over all i s.t. i is a state
aNum(i,j)=Sum of DiGamma(t,i,j) over all t's s.t. 0<=t<T-1
aDenom(i)=Sum of DiGamma(t,i,k) over all t's and all k's s.t. 0<=t<T-1, 0<=k<=N-1
"""
def computeDiGammaNum(t,i,j,alphaDP,detaDP,a,b,observations):
    return alphaDP[t][i]*a[i][j]*b[j][observations[t+1]]*betaDP[t+1][j]
def computeDiGammaDP(alphaDP,betaDP,a,b,observations):
    observationsC=alphaDP.shape[0]
    statesC=alphaDP.shape[1]
    diGammaDP=np.ndarray(shape=(statesC,statesC),dtype=float)
    diGammaDenom=observationsLikelihood(alphaDP)
    for i in np.arange(statesC):
        for j in np.arange(statesC):
                for t in np.arange(observationsC-1):
                    diGammaDP[i][j]+=(computeDiGammaNum(t,i,j,alphaDP,betaDP,a,b,observations)/diGammaDenom)
    return diGammaDP
def computeTransitionProbabilityA(alphaDP,betaDP,a,b,observations):
    statesC=alphaDP.shape[1]
    newlyComputedTransitionProbA=np.ndarray(shape=(statesC,statesC),dtype=float)
    diGammaDP=computeDiGammaDP(alphaDP,betaDP,a,b,observations)
    diGammaDPSumGrpByJ=np.ndarray(shape=(statesC),dtype=float)
    for i in np.arange(statesC):
        for j in np.arange(statesC):
            diGammaDPSumGrpByJ[i]+=diGammaDP[i][j]
    for i in np.arange(statesC):    
        for j in np.arange(statesC):
            newlyComputedTransitionProbA[i][j]=diGammaDP[i][j]/diGammaDPSumGrpByJ[i]
    return newlyComputedTransitionProbA   

In [17]:
#Compute Observation Probability: b[i][O[k]]~ b[i][vk] i.e. the Probability of observing the symbol vk at state i
"""
b[i][vk]=Expected no of times in state i and observing vk/Expected no of times in State i
            =ObsrProbNum(i,vk)/ObsrProbDenom(i)
gamma[t][j]=Probability of being in state j at time t
gamma[t][j]=p(q[t]=j|O,λ)=p(q[t]=j,O|λ)/p(O|λ)=gammaNum(t,j)/gammaDenom()
gammaNum(t,j)=p(q[t]=j,O|λ)=alphaDP[t][j]*betaDP[t][j]
gammaDenom()=observationsLikelihood(alphaDP) i.e. Sum of alphaDP[t-1][i] over all i s.t. i is a state
obsrProbDenom(i)= Sum of gamma[t][i] over all t s.t. 0<=t<T
obsrProbNum(i,vk)=Sum of gamma[t][i] over all t s.t. 0<=t<T and O[t]=vk
"""
def computeGammaNum(t,j,alphaDP,detaDP):
    return alphaDP[t][j]*betaDP[t][j]
def computeGammaDP(alphaDP,betaDP):
    observationsC=alphaDP.shape[0]
    statesC=alphaDP.shape[1]
    gammaDP=np.ndarray(shape=(statesC,observationsC),dtype=float)
    gammaDenom=observationsLikelihood(alphaDP)
    for i in np.arange(statesC):
        for t in np.arange(observationsC):
            gammaDP[i][t]=computeGammaNum(t,i,alphaDP,betaDP)/gammaDenom  
    return gammaDP
def computeObsrProbNum(gammaDP,i,vk,observations):
    observationC=len(observations)
    obsrProbNum=0.0
    for t in np.arange(observationC):
        if observations[t]==vk:
            obsrProbNum+=gammaDP[i][t]
    return obsrProbNum
def computeTransitionProbabilityB(alphaDP,betaDP,a,b,observations):
    statesC=a.shape[0]
    observationsC=b.shape[1]
    newlyComputedObsrProbB=np.ndarray(shape=(statesC,observationsC),dtype=float)
    gammaDP=computeGammaDP(alphaDP,betaDP) 
    for i in np.arange(statesC):
        obsrProbDenom =np.sum(gammaDP[i])
        for vk in observations:
            newlyComputedObsrProbB[i][vk]=computeObsrProbNum(gammaDP,i,vk,observations)/obsrProbDenom
    return newlyComputedObsrProbB   

In [18]:
observations=np.array([0,1,0,2,1,0])
A=np.array([[0.7,0.3],[0.4,0.6]])
B=np.array([[0.1,0.4,0.5],[0.7,0.2,0.1]])
pi=np.array([0.6,0.4])
alphaDP=np.zeros(shape=(observations.shape[0],A.shape[0]))# Count_of_Observations*Count_of_Hidden_States
betaDP=np.zeros(shape=(observations.shape[0],A.shape[0]))# Count_of_Observations*Count_of_Hidden_States
computeAlpha(observations,A,B,pi,alphaDP)
computeBeta(observations,A,B,pi,betaDP)
print(computeTransitionProbabilityA(alphaDP,betaDP,A,B,observations))
print(computeTransitionProbabilityB(alphaDP,betaDP,A,B,observations))

[[ 0.42772562  0.37468667]
 [ 0.38746073  0.47755799]]
[[ 0.23425023  0.45724387  0.3085059 ]
 [ 0.70813797  0.23628532  0.05557671]]


In [21]:
#Change Convergence Criteria to be more reasonable/Useful
def isConverged(count):
    if count>=5:
        return True
    return False
def Forward_Backward_EM_Algo(observations,A,B,pi):
    count=0
    while isConverged(count)==False:
        print("State Transition Matrix (A) ===>")
        print(A)
        print("Observation Probability Matrix (B) ===>")
        print(B)
        #Expectation(E)-Step
        alphaDP=np.zeros(shape=(observations.shape[0],A.shape[0]))# Count_of_Observations*Count_of_Hidden_States
        betaDP=np.zeros(shape=(observations.shape[0],A.shape[0]))# Count_of_Observations*Count_of_Hidden_States
        computeAlpha(observations,A,B,pi,alphaDP)
        computeBeta(observations,A,B,pi,betaDP)
        #Maximization(M)-Step
        newA=computeTransitionProbabilityA(alphaDP,betaDP,A,B,observations)
        newB=computeTransitionProbabilityB(alphaDP,betaDP,A,B,observations)
        A=newA
        B=newB
        count=count+1

In [20]:
observations=np.array([0,1,0,2,1,0])
A=np.array([[0.7,0.3],[0.4,0.6]])
B=np.array([[0.1,0.4,0.5],[0.7,0.2,0.1]])
pi=np.array([0.6,0.4])
Forward_Backward_EM_Algo(observations,A,B,pi)    

State Transition Matrix (A) ===>
[[ 0.7  0.3]
 [ 0.4  0.6]]
Observation Probability Matrix (B) ===>
[[ 0.1  0.4  0.5]
 [ 0.7  0.2  0.1]]
State Transition Matrix (A) ===>
[[ 0.53304969  0.46695031]
 [ 0.44792178  0.55207822]]
Observation Probability Matrix (B) ===>
[[ 0.23425023  0.45724387  0.3085059 ]
 [ 0.70813797  0.23628532  0.05557671]]
State Transition Matrix (A) ===>
[[ 0.45285207  0.54714793]
 [ 0.4825146   0.5174854 ]]
Observation Probability Matrix (B) ===>
[[ 0.38237174  0.37125943  0.24636883]
 [ 0.70945176  0.25178477  0.03876347]]
State Transition Matrix (A) ===>
[[ 0.4256403   0.5743597 ]
 [ 0.51099754  0.48900246]]
Observation Probability Matrix (B) ===>
[[ 0.50922725  0.27045219  0.22032056]
 [ 0.6781742   0.29022522  0.03160057]]
State Transition Matrix (A) ===>
[[ 0.40804421  0.59195579]
 [ 0.53939866  0.46060134]]
Observation Probability Matrix (B) ===>
[[ 0.614906    0.18607394  0.19902006]
 [ 0.61632299  0.35584658  0.02783043]]
